Baixar o arquivo csv, caso não tenha no projeto:

In [ ]:
import subprocess
import sys

# Tenta importar gdown, se não conseguir, instala
try:
    import gdown
except ImportError:
    print("📥 gdown não encontrado. Instalando automaticamente...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "gdown"])
    import gdown  # Tenta importar novamente após a instalação

# Baixar o arquivo do Google Drive

file_id = "1VbB_fvowkgNoWcLADi1mEfFjWQI6nBjp"
url = f"https://drive.google.com/uc?id={file_id}"
output = "dados/SpSafe_2022.csv"

gdown.download(url, output, quiet=False)


FileNotFoundError: [WinError 3] O sistema não pode encontrar o caminho especificado: 'dados'

Leitura do arquivo

In [1]:
import pandas as pd
df = pd.read_csv('dados/SpSafe_2022.csv', delimiter=';')


C:\Users\mathe\AppData\Local\Temp\ipykernel_26308\135612657.py:2: DtypeWarning: Columns (10,11,16,17,19,20,21,22,24,25,26,27,28,29,30,31,32,35,36,38,39,41,42,43,44,45,46,47,48,49) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('dados/SpSafe_2022.csv', delimiter=';')


In [3]:
df

,NUM_BO,ANO_BO,CODIGO_BOLETIM,NATUREZA_APURADA,DATA_OCORRENCIA,HORA_OCORRENCIA,PERIODO_OCORRENCIA,CIDADE,LOGRADOURO,NUMERO_LOGRADOURO,...,DATA_HORA_ELABORACAO,DATA_COMUNICACAO,BO_AUTORIA,FLAGRANTE,EXAME,SOLUCAO,ESPECIE,STATUS,FLAG_VITIMA_FATAL,DESDOBRAMENTO
0,21.0,2022,21/2022,HOMICIDIO DOLOSO,2021-12-31 00:00:00,15:40:00,A TARDE,NaN,DA VEDACAO DA DIVULGACAO DOS DADOS RELATIVOS,NaN,...,2021-12-31 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,312.0,2022,312/2022,HOMICIDIO DOLOSO,2022-01-20 00:00:00,07:00:00,PELA MANHA,NaN,PRACA DA BANDEIRA,10.0,...,2022-01-20 09:45:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,253.0,2022,253/2022,HOMICIDIO DOLOSO,2022-01-16 00:00:00,05:48:00,DE MADRUGADA,NaN,RUA PRATES,536.0,...,2022-01-16 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,305.0,2022,305/2022,HOMICIDIO DOLOSO,2022-01-22 00:00:00,19:50:00,NaN,NaN,RUA GUARANI,427.0,...,2022-01-22 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6.0,2022,6/2022,HOMICIDIO DOLOSO,2022-01-01 00:00:00,NaN,EM HORA INCERTA,NaN,PRACA DA REPUBLICA,9.0,...,2022-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
720441,4647.0,2022,4647/2022,LOCALIZACAO/APREENSAO DE VEICULO,2022-12-15 00:00:00,00:10:00,DE MADRUGADA,NaN,RUA JOAO BOEMER,499.0,...,2022-12-29 23:18:37,2022-12-15,DESCONHECIDA,NAO,NaN,APRECIACAO DO DELEGADO TITULAR,LOCALIZACAO E/OU DEVOLUCAO,CONSUMADO,NaN,NaN
720442,4647.0,2022,4647/2022,LOCALIZACAO/APREENSAO DE VEICULO,2022-12-15 00:00:00,00:10:00,DE MADRUGADA,NaN,RUA JOAO BOEMER,499.0,...,2022-12-29 23:18:37,2022-12-15,DESCONHECIDA,NAO,NaN,APRECIACAO DO DELEGADO TITULAR,LOCALIZACAO E/OU DEVOLUCAO,CONSUMADO,NaN,NaN
720443,4647.0,2022,4647/2022,ROUBO (ART. 157) - VEICULO,2022-12-15 00:00:00,00:10:00,DE MADRUGADA,NaN,RUA JOAO BOEMER,499.0,...,2022-12-29 23:18:37,2022-12-15,DESCONHECIDA,NAO,NaN,APRECIACAO DO DELEGADO TITULAR,TITULO II - PATRIMONIO (ARTS. 155 A 183),CONSUMADO,NaN,NaN
720444,4647.0,2022,4647/2022,ROUBO (ART. 157) - VEICULO,2022-12-15 00:00:00,00:10:00,DE MADRUGADA,NaN,RUA JOAO BOEMER,499.0,...,2022-12-29 23:18:37,2022-12-15,DESCONHECIDA,NAO,NaN,APRECIACAO DO DELEGADO TITULAR,TITULO II - PATRIMONIO (ARTS. 155 A 183),CONSUMADO,NaN,NaN


In [18]:
df.head()

,NUM_BO,ANO_BO,CODIGO_BOLETIM,NATUREZA_APURADA,DATA_OCORRENCIA,HORA_OCORRENCIA,PERIODO_OCORRENCIA,CIDADE,LOGRADOURO,NUMERO_LOGRADOURO,...,DATA_HORA_ELABORACAO,DATA_COMUNICACAO,BO_AUTORIA,FLAGRANTE,EXAME,SOLUCAO,ESPECIE,STATUS,FLAG_VITIMA_FATAL,DESDOBRAMENTO
0,21.0,2022,21/2022,HOMICIDIO DOLOSO,2021-12-31 00:00:00,15:40:00,A TARDE,NaN,DA VEDACAO DA DIVULGACAO DOS DADOS RELATIVOS,NaN,...,2021-12-31 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,312.0,2022,312/2022,HOMICIDIO DOLOSO,2022-01-20 00:00:00,07:00:00,PELA MANHA,NaN,PRACA DA BANDEIRA,10.0,...,2022-01-20 09:45:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,253.0,2022,253/2022,HOMICIDIO DOLOSO,2022-01-16 00:00:00,05:48:00,DE MADRUGADA,NaN,RUA PRATES,536.0,...,2022-01-16 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,305.0,2022,305/2022,HOMICIDIO DOLOSO,2022-01-22 00:00:00,19:50:00,NaN,NaN,RUA GUARANI,427.0,...,2022-01-22 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6.0,2022,6/2022,HOMICIDIO DOLOSO,2022-01-01 00:00:00,NaN,EM HORA INCERTA,NaN,PRACA DA REPUBLICA,9.0,...,2022-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
